In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import tensorflow_hub as hub
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras import layers, Model


from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
print(tf.__version__)

2.5.0


In [4]:
import pathlib
data_dir = pathlib.Path('gdrive/MyDrive/datasets/amazon_review_full_csv') 

In [6]:
sample_file = os.path.join(data_dir, 'readme.txt')
with open(sample_file) as f:
  print(f.read())


Amazon Review Full Score Dataset

Version 3, Updated 09/09/2015

ORIGIN

The Amazon reviews dataset consists of reviews from amazon. The data span a period of 18 years, including ~35 million reviews up to March 2013. Reviews include product and user information, ratings, and a plaintext review. For more information, please refer to the following paper: J. McAuley and J. Leskovec. Hidden factors and hidden topics: understanding rating dimensions with review text. RecSys, 2013.

The Amazon reviews full score dataset is constructed by Xiang Zhang (xiang.zhang@nyu.edu) from the above dataset. It is used as a text classification benchmark in the following paper: Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015).


DESCRIPTION

The Amazon reviews full score dataset is constructed by randomly taking 600,000 training samples and 130,000 testing samples for each review score from 

In [7]:
train_df = pd.read_csv('gdrive/MyDrive/datasets/amazon_review_full_csv/train/train.csv', names=["Rating","Review_Title","Review_Text"], index_col=False, delimiter=",")

train_df.head()

,Rating,Review_Title,Review_Text
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [8]:
Y = train_df["Rating"].to_numpy(np.int32) 
X = train_df["Review_Text"].to_numpy(dtype=object)


In [9]:
# Lenght of the train dataset
print(len(X))

3000000


In [10]:
#Creating a subset for simpler training process
data_length = len(X)
idx = np.random.permutation(data_length)
X_perm = X[idx]
Y_perm = Y[idx]

cut_ratio = 0.2
subdata_len = int( cut_ratio * data_length )

# Input and output divided into train and validation
x_sub = X_perm[:subdata_len]
y_sub = Y_perm[:subdata_len]


print("x_sub: {} len: {}".format( x_sub.shape, len(x_sub)))
print("y_sub: {} len: {}".format( y_sub.shape, len(y_sub)))


x_sub: (600000,) len: 600000
y_sub: (600000,) len: 600000


In [15]:
print(x_sub[30002])
print(y_sub[30002])

I've had this bag for 4 months now. It is a nice looking bag, and also has a nice light and small design. It fits my notebook, power chords, and notebook-mouse just fine without looking stuffed or too thick. In fact, I really liked this bag until one day...It was a rainy day. I packed my notebook as usual, stepped outside assuming all would be protected and safe. However, if you look carefully, the flap that swings over the top is not made to wrap around the edges. There is an angled gap of roughly half an inch on both sides of the flap, that provides direct access to the notebook.Luckily for me, it was a short walk to the car in very heavy rain. when I opened my notebook bag, I discovered water on the edges of my notebook. And now, I have to always remember to stuff paper towels on the edges just in case it rains.
2


In [16]:
#Split the Sub dataframe to train and validation.
train_ratio = 0.8
x_sub_lenght = len(x_sub)
idx = np.random.permutation(x_sub_lenght)

x_sub = x_sub[idx]
y_sub = y_sub[idx]

train_data_len = int( train_ratio * x_sub_lenght  )

x_train = x_sub[:train_data_len]
y_train = y_sub[:train_data_len]

x_val = x_sub[train_data_len:]
y_val = y_sub[train_data_len:]

print("x_train: {} len: {}".format( x_train.shape, len(x_train)))
print("y_train: {} len: {}".format( y_train.shape, len(y_train)))

print("x_val: {} len: {}".format( x_val.shape, len(x_val)))
print("y_val: {} len: {}".format( y_val.shape, len(y_val)))


x_train: (480000,) len: 480000
y_train: (480000,) len: 480000
x_val: (120000,) len: 120000
y_val: (120000,) len: 120000


In [17]:
# We define how the input layers to our model look like. One input is text and the other if the purchase is verified or not
#This shape means input is a tf.tensor and its shape is (None,)
input_text = layers.Input(shape=[], dtype=tf.string)
print(input_text)


KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.string, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [20]:
# We define how the input layers to our model look like. One input is text and the other if the purchase is verified or not
#This shape means input is a tf.tensor and its shape is (None,)
input_text = layers.Input(shape=[], dtype=tf.string)
print(input_text.shape)

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)


x1 = hub_layer(input_text)
#array = np.ones( (x.shape[0], 1) )
# combined = concatenate([x], axis=1)
# Shape is (None, 51) which is the 50 embedding representation + 1 from verified_purchase!
# print(combined.shape)
x = Dense(16, activation='relu')(x1)
x = Dense(1, activation='sigmoid')(x)

model_classifier = Model(inputs=input_text, outputs=x, name='FakeReviewDetector')

#Sanity check if our model structure is correct
model_classifier.summary()


(None,)
Model: "FakeReviewDetector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer_2 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 16)                416       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 48,192,308
Trainable params: 48,192,308
Non-trainable params: 0
_________________________________________________________________


In [21]:
model_classifier.compile(optimizer='adam',
                loss=tf.losses.BinaryCrossentropy(from_logits=True),
                metrics=[tf.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])


callbacks = [
    keras.callbacks.EarlyStopping(monitor="accuracy", patience=15,
                                    verbose=1, mode="min", restore_best_weights=True),
        keras.callbacks.ModelCheckpoint(filepath="Models/fakeNewsDetector_Advanced.hdf5", verbose=1, save_best_only=True)
    ]

history = model_classifier.fit(
                        x_train,
                        y_train,
                        epochs=5,
                        batch_size=1024,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        callbacks=callbacks
                        )

Epoch 1/5
469/469 [==============================] - 25s 45ms/step - loss: -25673.5254 - accuracy: 0.1947 - val_loss: -136984.4531 - val_accuracy: 0.2013

Epoch 00001: val_loss improved from inf to -136984.45312, saving model to Models/fakeNewsDetector_Advanced.hdf5
Epoch 2/5
469/469 [==============================] - 20s 43ms/step - loss: -875825.5000 - accuracy: 0.1994 - val_loss: -2236536.0000 - val_accuracy: 0.2013

Epoch 00002: val_loss improved from -136984.45312 to -2236536.00000, saving model to Models/fakeNewsDetector_Advanced.hdf5
Epoch 3/5
469/469 [==============================] - 21s 44ms/step - loss: -5566053.0000 - accuracy: 0.1994 - val_loss: -10257549.0000 - val_accuracy: 0.2013

Epoch 00003: val_loss improved from -2236536.00000 to -10257549.00000, saving model to Models/fakeNewsDetector_Advanced.hdf5
Epoch 4/5
469/469 [==============================] - 21s 44ms/step - loss: -18468398.0000 - accuracy: 0.1994 - val_loss: -28780230.0000 - val_accuracy: 0.2013

Epoch 000